In [2]:
#Import dependenies
import os
import matplotlib.ticker as ticker
import matplotlib.cm as cm
import matplotlib as mpl
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from pandas.io.json import json_normalize
import requests
import json
from gmplot import gmplot
import time
import numpy as np
import folium as fl
from yelpapi import YelpAPI
from pprint import pprint
from API import api_key
import seaborn as sns
os.environ['PROJ_LIB'] = "/anaconda3/envs/test/share/proj"

ModuleNotFoundError: No module named 'API'

In [ ]:
#Get information using Yelp API from Yelp
url = 'https://api.yelp.com/v3/businesses/search'
offset = 0
API = api_key
headers = {'Authorization': 'bearer %s' % API}
business = []

for i in range(0, 500, 50):
    params = {'location': 'Cleveland, Ohio', 'offset': offset, 'limit': 50}
    response = requests.get(url, params=params, headers=headers)
    business.append(response.json())
    offset += 50
    
    if offset % 50 == 0:
        time.sleep(5)
        
business

In [ ]:
#Create a dataframe
response = business[0]['businesses']
df = pd.DataFrame.from_dict(response)
df.head()

In [ ]:
#Concat all the dataframes into one
full_data = []

for i in business:
    full_data = full_data + i['businesses']

df = pd.DataFrame.from_dict(full_data)
df.head()

In [ ]:
#Clean the dataframe from dictionaries
idx = 0
lats = []
lngs = []
Bus_titles = []
ZIPS = []
for i, r in df.iterrows():
    lat = r["coordinates"]["latitude"]
    lng = r["coordinates"]["longitude"]
    titles = r["categories"][0]["title"]
    ZIP = r["location"]["zip_code"]
    lats.append(lat)
    lngs.append(lng)
    Bus_titles.append(titles)
    ZIPS.append(ZIP)
    
clean_df = pd.DataFrame(data={'Categories': Bus_titles, 'latitude': lats,'longitude':lngs, 'zip_code': ZIPS})
clean_df.head()

In [ ]:
#Add the cleaned columns into datframe and drop the unnecessary columns
concat_df = pd.concat([df,clean_df], axis=1)

#Drop the columns
yelp = concat_df.drop(columns = ['coordinates', 'categories', 'location'])
yelp.head()

In [ ]:

grouped_yelp = yelp.groupby(['Categories'], as_index=False).sum()
review = grouped_yelp.sort_values("review_count", ascending=False)
review.head()


In [ ]:
#Use a plot to visualize the frequency of rating
sns.countplot(x='rating',data=yelp);

#we can see that most of the businesses in cleveland had got a rating of 4.0 with a review count of more than 200.

In [2]:
from wordcloud import WordCloud, STOPWORDS
plt.figure(figsize=(12,10))

wordcloud = WordCloud(background_color='white',
                          width=1200,
                      stopwords = STOPWORDS,
                          height=1000
                         ).generate(str(yelp['Categories']))


plt.imshow(wordcloud)
plt.axis('off');


#top businesses on our dataset. The height of each word in this picture is an indication of frequency of occurrence of the word in the entire text.
#words like mexican, american, bars, coffee and tea are frequently used in the reviews.

NameError: name 'yelp' is not defined

<Figure size 864x720 with 0 Axes>

In [ ]:
plt.figure(figsize=(20,12))
f = sns.barplot( y= 'review_count',x = 'Categories' , data = review.iloc[0:10])
f.set_title('Top 5 Categories with highest Review Count')
f.set_ylabel('Review Count')
f.set_xlabel('Category');

In [3]:
gmap = gmplot.GoogleMapPlotter(41.4, -81.7, 13)
#gmap = gmplot.GoogleMapPlotter.from_geocode("Cleveland")
gmap.scatter(yelp['latitude'], yelp['longitude'], '#3B0B39', size=40, marker=False)
gmap.draw("my_map.html")
gmap

NameError: name 'yelp' is not defined